In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Carregar os dados
df = pd.read_excel('/content/dataset.xlsx', header=0)

# Selecionar colunas (DEIXA 'NETPRO  ' PQ NO DATASET TA ASSIM)
cols = ['NETPRO  ', 'Q20Age', 'Q21Gender', 'Q22Income', 'Q23FLY', 'Q5TIMESFLOWN', 'Q6LONGUSE']
# Limpa valores nulos pra não influenciar (dropna)
data = df[cols].dropna().copy()

# Normalizar os dados
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Aplicar DBSCAN para detectar ruídos (cluster = -1)
# Usamos DBSCAN ao invés de Kmeans porque não sabia definir a quantidade de clusters
# configs (eps = distância um do outro para serem agrupados) (usamos 1.5)
# configs (min_samples = 'número mínimo de linhas para formar um cluster') (usamos 10)
dbscan = DBSCAN(eps=1.5, min_samples=10)
labels = dbscan.fit_predict(data_scaled)

# Adiciona o número do cluster à tabela
data['cluster'] = labels

# Calcular percentual de ruído (ruídos = -1)
noise_pct = (labels == -1).sum() / len(labels) * 100
print(f'Percentual de passageiros considerados outliers: {noise_pct:.2f}%')

# Gerar perfil dos clusters (exceto ruído)
cluster_profile = data[data['cluster'] != -1].groupby('cluster').mean()
print("\nPerfil médio dos clusters (sem outliers):")
print(cluster_profile)

# Perfil dos outliers
outlier_profile = data[data['cluster'] == -1].mean(numeric_only=True)
print("\nPerfil médio dos outliers:")
print(outlier_profile)

# Exibir mapeamento de gênero
print("\nMapeamento de código de gênero:")
print("{0: 'Masculino', 1: 'Feminino', 2: 'Outro', 3: 'Prefiro não informar'}")

Percentual de passageiros considerados outliers: 8.26%

Perfil médio dos clusters (sem outliers):
          NETPRO      Q20Age  Q21Gender  Q22Income    Q23FLY  Q5TIMESFLOWN  \
cluster                                                                      
0         8.221429  4.246429        1.0   2.424107  1.910714      2.249107   
1         8.408472  4.253404        2.0   2.025719  1.989410      2.204992   
2        10.302521  0.033613        0.0   0.000000  0.000000      2.361345   
3         8.250000  2.625000        2.0   0.062500  0.000000      1.312500   

         Q6LONGUSE  
cluster             
0         2.559821  
1         2.605144  
2         2.386555  
3         1.250000  

Perfil médio dos outliers:
NETPRO          6.254310
Q20Age          4.047414
Q21Gender       1.176724
Q22Income       1.534483
Q23FLY          1.439655
Q5TIMESFLOWN    2.439655
Q6LONGUSE       2.538793
cluster        -1.000000
dtype: float64

Mapeamento de código de gênero:
{0: 'Masculino', 1: 'Feminino',

a. Existe um grupo incomum de passageiros que não se enquadra no perfil típico de cliente do
aeroporto?

Sim, o modelo identificou 8.26% dos passageiros como outliers (ruídos) usando o algoritmo DBSCAN. Esses passageiros não se enquadram no perfil típico do aeroporto.

Qual é o tamanho do cluster em percentagem aos passageiros do aeroporto e qual é o perfil
do grupo?

8,26% dos passageiros foram classificados como fora do padrão típico, ou seja, não pertencem a nenhum dos clusters principais identificados.

Perfil médio desse grupo incomum:

    Satisfação (NETPRO): 6,25 (significativamente mais baixa que os outros clusters, que variam de 8,2 a 10,3)

    Idade (Q20Age): 4,05 (faixa etária média-alta)

    Gênero (Q21Gender): 1,18 (predominância levemente maior de mulheres ou outros gêneros, dependendo da codificação original)

    Renda (Q22Income): 1,53 (renda relativamente baixa)

    Frequência de voo (Q23FLY): 1,44 (viajam pouco)

    Experiência de voos (Q5TIMESFLOWN): 2,44 (experiência semelhante à média)

    Tempo de uso do aeroporto (Q6LONGUSE): 2,54 (usuários com algum histórico, mas não longos frequentadores)